In [73]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
'C:\Users\David' is not recognized as an internal or external command,
operable program or batch file.


In [43]:
import os.path
from pprint import pprint
import time
from io import BytesIO
from random import random
import uuid

from azure.cognitiveservices.vision.contentmoderator import ContentModeratorClient
import azure.cognitiveservices.vision.contentmoderator.models
from msrest.authentication import CognitiveServicesCredentials

In [31]:
CONTENTMODERATOR_ENDPOINT = "<your API endpoint>"
SUBSCRIPTION_KEY = "<your subscription key"

Authenticate the client

In [37]:
client = ContentModeratorClient(
    endpoint=CONTENTMODERATOR_ENDPOINT,
    credentials=CognitiveServicesCredentials(SUBSCRIPTION_KEY)
)

## Moderate Text

In [41]:
TEXT_FOLDER = os.path.join(os.path.dirname(
    os.path.abspath('')), "azure-cognitive-services-content-moderator-python/text_files")

In [44]:
# Screen the input text: check for profanity,
# do autocorrect text, and check for personally identifying
# information (PII)
with open(os.path.join(TEXT_FOLDER, 'content_moderator_text_moderation.txt'), "rb") as text_fd:
    screen = client.text_moderation.screen_text(
        text_content_type="text/plain",
        text_content=text_fd,
        language="eng",
        autocorrect=True,
        pii=True
    )
    #assert isinstance(screen, Screen)
    pprint(screen.as_dict())

{'auto_corrected_text': 'Is this a garbage email abcdef@abcd.com, phone: '
                        '4255550111, IP: 255.255.255.255, 1234 Main Boulevard, '
                        'Pentapolis WA 96555.\r\n'
                        'Crap is the profanity here. Is this information PII? '
                        'phone 4255550111',
 'language': 'eng',
 'normalized_text': '   grabage email abcdef@abcd.com, phone: 4255550111, IP: '
                    '255.255.255.255, 1234 Main Boulevard, Panapolis WA '
                    '96555.\r\n'
                    'Crap   profanity .   information PII? phone 4255550111',
 'original_text': 'Is this a grabage email abcdef@abcd.com, phone: 4255550111, '
                  'IP: 255.255.255.255, 1234 Main Boulevard, Panapolis WA '
                  '96555.\r\n'
                  'Crap is the profanity here. Is this information PII? phone '
                  '4255550111',
 'pii': {'address': [{'index': 81,
                      'text': '1234 Main Boulevar

### Use a custom terms list

In [52]:
LATENCY_DELAY = 1
TEXT_FOLDER = os.path.join(os.path.dirname(
    os.path.realpath('')), "azure-cognitive-services-content-moderator-python/text_files")

In [54]:
#
# Create list
#
print("\nCreating list")
custom_list = client.list_management_term_lists.create(
    content_type="application/json",
    body={
        "name": "Term list name",
        "description": "Term list description",
    }
)
print("List created:")
#assert isinstance(custom_list, TermList)
pprint(custom_list.as_dict())
list_id = custom_list.id

#
# Update list details
#
print("\nUpdating details for list {}".format(list_id))
updated_list = client.list_management_term_lists.update(
    list_id=list_id,
    content_type="application/json",
    body={
        "name": "New name",
        "description": "New description"
    }
)
#assert isinstance(updated_list, TermList)
pprint(updated_list.as_dict())

#
# Add terms
#
print("\nAdding terms to list {}".format(list_id))
client.list_management_term.add_term(
    list_id=list_id,
    term="term1",
    language="eng"
)
client.list_management_term.add_term(
    list_id=list_id,
    term="term2",
    language="eng"
)

#
# Get all terms ids
#
print("\nGetting all term IDs for list {}".format(list_id))
terms = client.list_management_term.get_all_terms(
    list_id=list_id, language="eng")
#assert isinstance(terms, Terms)
terms_data = terms.data
#assert isinstance(terms_data, TermsData)
pprint(terms_data.as_dict())

#
# Refresh the index
#
print("\nRefreshing the search index for list {}".format(list_id))
refresh_index = client.list_management_term_lists.refresh_index_method(
    list_id=list_id, language="eng")
#assert isinstance(refresh_index, RefreshIndex)
pprint(refresh_index.as_dict())

print("\nWaiting {} minutes to allow the server time to propagate the index changes.".format(
    LATENCY_DELAY))
time.sleep(LATENCY_DELAY * 60)

#
# Screen text
#
with open(os.path.join(TEXT_FOLDER, 'content_moderator_term_list.txt'), "rb") as text_fd:
    screen = client.text_moderation.screen_text(
        text_content_type="text/plain",
        text_content=text_fd,
        language="eng",
        autocorrect=False,
        pii=False,
        list_id=list_id
    )
    #assert isinstance(screen, Screen)
    pprint(screen.as_dict())

#
# Remove terms
#
term_to_remove = "term1"
print("\nRemove term {} from list {}".format(term_to_remove, list_id))
client.list_management_term.delete_term(
    list_id=list_id,
    term=term_to_remove,
    language="eng"
)

#
# Delete all terms
#
print("\nDelete all terms in the image list {}".format(list_id))
client.list_management_term.delete_all_terms(
    list_id=list_id, language="eng")


Creating list
List created:
{'description': 'Term list description', 'id': 163, 'name': 'Term list name'}

Updating details for list 163
{'description': 'New description', 'id': 163, 'name': 'New name'}

Adding terms to list 163

Getting all term IDs for list 163
{'language': 'eng',
 'status': {'code': 3000, 'description': 'OK'},
 'terms': [{'term': 'term1'}, {'term': 'term2'}],
 'tracking_id': 'USW2_ibiza_5b2b8551-c418-48bf-ab44-67074a66f136_ContentModerator.F0_c208c5ec-4c10-4bc9-8a9d-35501c4483ff'}

Refreshing the search index for list 163
{'advanced_info': [],
 'content_source_id': '163',
 'is_update_success': True,
 'status': {'code': 3000, 'description': 'OK'},
 'tracking_id': 'USW2_ibiza_5b2b8551-c418-48bf-ab44-67074a66f136_ContentModerator.F0_b1f54db8-7ef2-4cbf-abcd-7e96a5e6ab85'}

Waiting 1 minutes to allow the server time to propagate the index changes.
{'language': 'eng',
 'normalized_text': ' text contains  terms "term1"  "term2".',
 'original_text': 'This text contains the

## Moderate images

In [55]:
IMAGE_LIST = [
    "https://moderatorsampleimages.blob.core.windows.net/samples/sample2.jpg",
    "https://moderatorsampleimages.blob.core.windows.net/samples/sample5.png"
]

In [56]:
for image_url in IMAGE_LIST:
    print("\nEvaluate image {}".format(image_url))


Evaluate image https://moderatorsampleimages.blob.core.windows.net/samples/sample2.jpg

Evaluate image https://moderatorsampleimages.blob.core.windows.net/samples/sample5.png


In [57]:
print("\nEvaluate for adult and racy content.")
evaluation = client.image_moderation.evaluate_url_input(
    content_type="application/json",
    cache_image=True,
    data_representation="URL",
    value=image_url
)
#assert isinstance(evaluation, Evaluate)
pprint(evaluation.as_dict())


Evaluate for adult and racy content.
{'adult_classification_score': 0.0013530384749174118,
 'advanced_info': [{'key': 'ImageDownloadTimeInMs', 'value': '548'},
                   {'key': 'ImageSizeInBytes', 'value': '2278902'}],
 'cache_id': '66fc26c3-77f6-43c2-97e5-ac17855426f0_637416664972918935',
 'is_image_adult_classified': False,
 'is_image_racy_classified': False,
 'racy_classification_score': 0.0045312270522117615,
 'result': False,
 'status': {'code': 3000, 'description': 'OK'},
 'tracking_id': 'USW2_ibiza_5b2b8551-c418-48bf-ab44-67074a66f136_ContentModerator.F0_13240c96-2976-42a9-9e93-72027225b98c'}


In [58]:
print("\nDetect and extract text.")
evaluation = client.image_moderation.ocr_url_input(
    language="eng",
    content_type="application/json",
    data_representation="URL",
    value=image_url,
    cache_image=True,
)
#assert isinstance(evaluation, OCR)
pprint(evaluation.as_dict())


Detect and extract text.
{'candidates': [],
 'language': 'eng',
 'metadata': [{'key': 'ImageDownloadTimeInMs', 'value': '477'},
              {'key': 'ImageSizeInBytes', 'value': '2278902'}],
 'status': {'code': 3000, 'description': 'OK'},
 'text': '',
 'tracking_id': 'USW2_ibiza_5b2b8551-c418-48bf-ab44-67074a66f136_ContentModerator.F0_472d7fa2-767c-4438-b8a4-f4fb6c60b161'}


In [59]:
print("\nDetect faces.")
evaluation = client.image_moderation.find_faces_url_input(
    content_type="application/json",
    cache_image=True,
    data_representation="URL",
    value=image_url
)
#assert isinstance(evaluation, FoundFaces)
pprint(evaluation.as_dict())


Detect faces.
{'advanced_info': [{'key': 'ImageDownloadTimeInMs', 'value': '149'},
                   {'key': 'ImageSizeInBytes', 'value': '2278902'}],
 'count': 6,
 'faces': [{'bottom': 598, 'left': 44, 'right': 268, 'top': 374},
           {'bottom': 620, 'left': 308, 'right': 532, 'top': 396},
           {'bottom': 575, 'left': 594, 'right': 773, 'top': 396},
           {'bottom': 563, 'left': 812, 'right': 955, 'top': 420},
           {'bottom': 611, 'left': 972, 'right': 1151, 'top': 432},
           {'bottom': 510, 'left': 1232, 'right': 1456, 'top': 286}],
 'result': True,
 'status': {'code': 3000, 'description': 'OK'},
 'tracking_id': 'USW2_ibiza_5b2b8551-c418-48bf-ab44-67074a66f136_ContentModerator.F0_556eb89f-45be-44fe-ad9c-97be635f6ea0'}


### Use a custom image list

In [62]:
IMAGE_LIST = {
    "Sports": [
        "https://moderatorsampleimages.blob.core.windows.net/samples/sample4.png",
        "https://moderatorsampleimages.blob.core.windows.net/samples/sample6.png",
        "https://moderatorsampleimages.blob.core.windows.net/samples/sample9.png"
    ],
    "Swimsuit": [
        "https://moderatorsampleimages.blob.core.windows.net/samples/sample1.jpg",
        "https://moderatorsampleimages.blob.core.windows.net/samples/sample3.png",
        "https://moderatorsampleimages.blob.core.windows.net/samples/sample4.png",
        "https://moderatorsampleimages.blob.core.windows.net/samples/sample16.png"
    ]
}

IMAGES_TO_MATCH = [
    "https://moderatorsampleimages.blob.core.windows.net/samples/sample1.jpg",
    "https://moderatorsampleimages.blob.core.windows.net/samples/sample4.png",
    "https://moderatorsampleimages.blob.core.windows.net/samples/sample5.png",
    "https://moderatorsampleimages.blob.core.windows.net/samples/sample16.png"
]

In [66]:
#
# Create list
#
print("Creating list MyList\n")
custom_list = client.list_management_image_lists.create(
    content_type="application/json",
    body={
        "name": "MyList",
        "description": "A sample list",
        "metadata": {
            "key_one": "Acceptable",
            "key_two": "Potentially racy"
        }
    }
)
print("List created:")
#assert isinstance(custom_list, ImageList)
pprint(custom_list.as_dict())
list_id = custom_list.id

#
# Add images
#
def add_images(list_id, image_url, label):
    """Generic add_images from url and label."""
    print("\nAdding image {} to list {} with label {}.".format(image_url, list_id, label))
    try:
        added_image = client.list_management_image.add_image_url_input(
            list_id=list_id,
            content_type="application/json",
            data_representation="URL",
            value=image_url,
            label=label
        )
    except APIErrorException as err:
        # sample4 will fail
        print("Unable to add image to list: {}".format(err))
    else:
        #assert isinstance(added_image, Image)
        pprint(added_image.as_dict())
        return added_image
print("\nAdding images to list {}".format(list_id))
index = {}  # Keep an index url to id for later removal
for label, urls in IMAGE_LIST.items():
    for url in urls:
        image = add_images(list_id, url, label)
        if image:
            index[url] = image.content_id

#
# Get all images ids
#
print("\nGetting all image IDs for list {}".format(list_id))
image_ids = client.list_management_image.get_all_image_ids(list_id=list_id)
#assert isinstance(image_ids, ImageIds)
pprint(image_ids.as_dict())

#
# Update list details
#
print("\nUpdating details for list {}".format(list_id))
updated_list = client.list_management_image_lists.update(
    list_id=list_id,
    content_type="application/json",
    body={
        "name": "Swimsuits and sports"
    }
)
#assert isinstance(updated_list, ImageList)
pprint(updated_list.as_dict())

#
# Get list details
#
print("\nGetting details for list {}".format(list_id))
list_details = client.list_management_image_lists.get_details(
    list_id=list_id)
#assert isinstance(list_details, ImageList)
pprint(list_details.as_dict())

#
# Refresh the index
#
print("\nRefreshing the search index for list {}".format(list_id))
refresh_index = client.list_management_image_lists.refresh_index_method(
    list_id=list_id)
#assert isinstance(refresh_index, RefreshIndex)
pprint(refresh_index.as_dict())

print("\nWaiting {} minutes to allow the server time to propagate the index changes.".format(
    LATENCY_DELAY))
time.sleep(LATENCY_DELAY * 60)

#
# Match images against the image list.
#
for image_url in IMAGES_TO_MATCH:
    print("\nMatching image {} against list {}".format(image_url, list_id))
    match_result = client.image_moderation.match_url_input(
        content_type="application/json",
        list_id=list_id,
        data_representation="URL",
        value=image_url,
    )
    #assert isinstance(match_result, MatchResponse)
    print("Is match? {}".format(match_result.is_match))
    print("Complete match details:")
    pprint(match_result.as_dict())

#
# Remove images
#
correction = "https://moderatorsampleimages.blob.core.windows.net/samples/sample16.png"
print("\nRemove image {} from list {}".format(correction, list_id))
client.list_management_image.delete_image(
    list_id=list_id,
    image_id=index[correction]
)

#
# Delete all images
#
print("\nDelete all images in the image list {}".format(list_id))
client.list_management_image.delete_all_images(list_id=list_id)

#
# Delete list
#
print("\nDelete the image list {}".format(list_id))
client.list_management_image_lists.delete(list_id=list_id)

Creating list MyList

List created:
{'description': 'A sample list',
 'id': 862635,
 'metadata': {'key_one': 'Acceptable', 'key_two': 'Potentially racy'},
 'name': 'MyList'}

Adding images to list 862635

Adding image https://moderatorsampleimages.blob.core.windows.net/samples/sample4.png to list 862635 with label Sports.
{'additional_info': [{'key': 'ImageDownloadTimeInMs', 'value': '468'},
                     {'key': 'ImageSizeInBytes', 'value': '2945548'},
                     {'key': 'Source', 'value': '862635'}],
 'content_id': '1030312',
 'status': {'code': 3000, 'description': 'OK'},
 'tracking_id': 'USW2_ibiza_5b2b8551-c418-48bf-ab44-67074a66f136_ContentModerator.F0_13a4a100-e85c-49b3-87a3-d502c3809c32'}

Adding image https://moderatorsampleimages.blob.core.windows.net/samples/sample6.png to list 862635 with label Sports.
{'additional_info': [{'key': 'ImageDownloadTimeInMs', 'value': '489'},
                     {'key': 'ImageSizeInBytes', 'value': '2440050'},
                

NameError: name 'APIErrorException' is not defined

## Create a review

In [71]:
# The name of the team to assign the job to.
# This must be the team name you used to create your Content Moderator account. You can
# retrieve your team name from the Review tool web site. Your team name is the Id
# associated with your subscription.
team_name = "<insert your team name here>"

# An image to review
image_url = "https://moderatorsampleimages.blob.core.windows.net/samples/sample5.png"

# Where you want to receive the approval/refuse event. This is the only way to get this information.
call_back_endpoint = "https://requestb.in/qmsakwqm"

In [72]:
# Create review
print("Create review for {}.\n".format(image_url))
review_item = {
    "type": "Image",             # Possible values include: 'Image', 'Text'
    "content": image_url,        # How to download the image
    "content_id": uuid.uuid4(),  # Random id
    "callback_endpoint": call_back_endpoint,
    "metadata": [{
        "key": "sc",
        "value": True  # will be sent to Azure as "str" cast.
    }]
}

reviews = client.reviews.create_reviews(
    url_content_type="application/json",
    team_name=team_name,
    create_review_body=[review_item]  # As many review item as you need
)

# Get review ID
review_id = reviews[0]  # Ordered list of string of review ID

print("\nGet review details")
review_details = client.reviews.get_review(
    team_name=team_name, review_id=review_id)
pprint(review_details.as_dict())

Create review for https://moderatorsampleimages.blob.core.windows.net/samples/sample5.png.


Get review details
{'callback_endpoint': 'https://southcentralus.contentmoderator.cognitive.microsoft.com/',
 'content': 'https://moderatorsampleimages.blob.core.windows.net/samples/sample5.png',
 'content_id': 'c9570395-1708-43f2-b386-0b6f52e51c86',
 'created_by': 'innovaccioncontentmoderator',
 'metadata': [{'key': 'sc', 'value': 'True'}],
 'review_id': '202011ic4b8ebb318834eebbcefc24aa0b26480',
 'reviewer_result_tags': [],
 'status': 'Pending',
 'sub_team': 'public',
 'type': 'Image'}
